# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#total gastado por cliente:
aggregate_amount_spent = orders.groupby(by = "CustomerID").agg({"amount_spent": "sum"})
#Cuantiles:
quantiles = aggregate_amount_spent.quantile([0.25, 0.50, 0.75, 0.95])
quantiles.head()

,amount_spent
0.25,307.245
0.50,674.450
0.75,1661.640
0.95,5840.182


In [4]:
#Clientes entre Q1 y Q2:
customers_Q1_Q2 = aggregate_amount_spent[(aggregate_amount_spent["amount_spent"] > 307.245) & (aggregate_amount_spent["amount_spent"] < 674.450)]
customers_Q1_Q2.head()

,amount_spent
CustomerID,
12350,334.40
12355,459.40
12363,552.00
12365,641.38
12373,364.60


In [5]:
#Clientes VIP:
customers_VIP = aggregate_amount_spent[aggregate_amount_spent["amount_spent"] > 5840.182]
customers_VIP = customers_VIP.sort_values("amount_spent", ascending = False) #ordeno para usar luego
customers_VIP.head()

,amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06


In [6]:
#Clientes preferentes:
customers_preferred = aggregate_amount_spent[(aggregate_amount_spent["amount_spent"] > 1661.64) & (aggregate_amount_spent["amount_spent"] < 5840.182)]
customers_preferred.sort_values("amount_spent", ascending = False).head() #ordeno para usar luego

,amount_spent
CustomerID,
13050,5836.86
12720,5781.73
15218,5756.89
17686,5739.46
13178,5725.47


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [7]:
#Clientes VIP:
customer_VIP = pd.DataFrame((customers_VIP.index))
customer_VIP.head()

,CustomerID
0,14646
1,18102
2,17450
3,16446
4,14911


In [8]:
#Los países de estos clientes:
lista = pd.DataFrame()
for value in range(217): #Filas de customer_VIP
    lista = pd.DataFrame(lista.append(list(orders["Country"][orders["CustomerID"] == customer_VIP["CustomerID"][value]].unique())))
lista.head()

,0
0,Netherlands
0,United Kingdom
0,United Kingdom
0,United Kingdom
0,EIRE


In [9]:
#Número de clientes VIP por país:
lista.columns = ["Country"]
lista = lista.reset_index()
lista.drop(["index"], axis='columns', inplace=True)
lista.groupby(by = "Country").agg({"Country" : ["count"]})

#UK es el que más clientes VIP tiene:

,Country
,count
Country,
Australia,2
Belgium,1
Channel Islands,1
Cyprus,1
Denmark,1
EIRE,2
Finland,1
France,9


In [10]:
#Clientes VIP+Preferred:
customer_VIP_Preferred = pd.DataFrame((customers_preferred.index))
customer_VIP_Preferred.head()

,CustomerID
0,12347
1,12348
2,12349
3,12352
4,12356


In [14]:
#Los países de estos clientes:
lista_ = pd.DataFrame()
for value in range(868): #Filas en customer_VIP_Preferred
    lista_ = pd.DataFrame(lista_.append(list(orders["Country"][orders["CustomerID"] == customer_VIP_Preferred["CustomerID"][value]].unique())))
lista_.head()

,0
0,Iceland
0,Finland
0,Italy
0,Norway
0,Portugal


In [16]:
#Número de clientes VIP_Preferred por país:
lista_.columns = ["Country"]
lista_ = lista_.reset_index()
lista_.drop(["index"], axis='columns', inplace=True)
lista_.groupby(by = "Country").agg({"Country" : ["count"]})

#UK es el que más clientes VIP_Preferred tiene:

,Country
,count
Country,
Australia,2
Austria,3
Belgium,11
Canada,1
Channel Islands,3
Cyprus,3
Denmark,2
EIRE,1
